$$ 没

In [1]:
using Revise
using Test

In [2]:
using LinearAlgebra
using ITensors
import CTMRG

$$一般の行列の対角化

In [3]:
new_1 = Index(2, "new = 1")
new_2 = Index(2, "new = 2")
old_1 = Index(2, "old = 1")
old_2 = Index(2, "old = 2")

(dim=2|id=81|"old=2")

In [4]:
C = ITensor(old_1, old_2)
C[old_1=>1, old_2=>1] = 2.0
C[old_1=>2, old_2=>1] = 1.0
C[old_1=>1, old_2=>2] = 1.0 
C[old_1=>2, old_2=>2] = 2.0
C_ = array(permute(C, (old_1, old_2)))
#array(permute(C, (old_2, old_1)))
@show evals, evecs = eigen(C_)
@show ievecs = inv(evecs)
evecs * ievecs
@show evecs[1,1]
;

(evals, evecs) = eigen(C_) = Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}([1.0, 3.0], [-0.7071067811865475 0.7071067811865475; 0.7071067811865475 0.7071067811865475])
ievecs = inv(evecs) = [-0.7071067811865476 0.7071067811865476; 0.7071067811865476 0.7071067811865476]
evecs[1, 1] = -0.7071067811865475


In [88]:
#固有値を移す
T = ITensor(new_1, new_2)
A = ITensor(old_1, new_1)
iA = ITensor(old_2, new_2)
for i in 1:2
    T[new_1=>i, new_2=>i] = evals[i]
end
T[new_1=>1, new_2=>1] = evals[1]
T[new_1=>2, new_2=>2] = evals[2]

3.0

In [94]:
for i in 1:2, j in 1:2
    A[old_1=>i,new_1 =>j] = evecs[i, j]
    iA[old_2=>i, new_2=> j] = ievecs[i, j]
end

In [190]:
#=
function make_diag_Tesnor(eigenvalues::Vector{Float64}, 
    eigenvec::Matrix{Float64}, ieigenvec::Matrix{Float64},
    L::Int)
    dim = length(eigenvalues)
    new1_ind = Index(dim, "eig_1")
    new2_ind = Index(dim, "eig_2")
    #固有値を移す
    Λ = ITensor(new1_ind, new2_ind)
    for i in 1:dim
        Λ[new1_ind => i, new2_ind => i] = eigenvalues[i]
    end
    #固有ベクトルを移す
    #A = ITensor(new1_ind, old1_ind_list...)
    #iA = ITensor(old2_ind_list..., new2_ind)
    for i in 1:dim, j in 1:dim
        A[old_1 => i,new1_ind => j] = evecs[i, j]
        iA[old_2 => i, new2_ind => j] = ievecs[i, j]
    end
    return Λ, A, iA
end
=#
function make_diag_Tesnor(eigenvalues::Vector{Float64}, 
    evecs::Matrix{Float64}, ievec::Matrix{Float64})
    dim = length(eigenvalues)
    old1_ind_temp = Index(dim, "old_1")
    old2_ind_temp = Index(dim, "old_2")
    new1_ind = Index(dim, "eig_1")
    new2_ind = Index(dim, "eig_2")
    #固有値を移す
    Λ = ITensor(new1_ind, new2_ind)
    for i in 1:dim
        Λ[new1_ind => i, new2_ind => i] = eigenvalues[i]
    end
    #固有ベクトルを移す
    A = ITensor(new1_ind, old1_ind_temp)
    iA = ITensor(old2_ind_temp, new2_ind)
    for i in 1:dim, j in 1:dim
        A[new1_ind => i, old1_ind_temp => j] = evecs[i, j]
        iA[old2_ind_temp => i, new2_ind => j] = ievecs[i, j]
    end
    return Λ, A, iA
end
    


make_diag_Tesnor (generic function with 4 methods)

In [191]:
new_1 = Index(2, "new = 1")
new_2 = Index(2, "new = 2")
new1_list =[new_1]
new2_list = [new_2]
C = [2.0 1.0; 1.0 2.0]
evals, evecs = eigen(C)
ievecs = inv(evecs)
Λ, A, iA = make_diag_Tesnor(evals, evecs, ievecs)
;

In [192]:
@show inds(A)[1]
@show inds(A)[2]
@show inds(iA)[1]
@show inds(iA)[2]
#規格化のチェック
_A = replaceinds(A, (inds(A)[1], inds(A)[2]) => (inds(A)[1], inds(iA)[1]))
@show inds(_A)[1]
@show inds(_A)[2]
@show  _A * iA 
;

(inds(A))[1] = (dim=2|id=418|"eig_1")
(inds(A))[2] = (dim=2|id=448|"old_1")
(inds(iA))[1] = (dim=2|id=513|"old_2")
(inds(iA))[2] = (dim=2|id=620|"eig_2")
(inds(_A))[1] = (dim=2|id=418|"eig_1")
(inds(_A))[2] = (dim=2|id=513|"old_2")
_A * iA = ITensor ord=2
Dim 1: (dim=2|id=418|"eig_1")
Dim 2: (dim=2|id=620|"eig_2")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 1.0  0.0
 0.0  1.0


In [ ]:
@testset "make_diag_Tensorのテスト" begin
    new_1 = Index(2, "new = 1")
    new_2 = Index(2, "new = 2")
    new1_list =[new_1]
    new2_list = [new_2]
    C = [2.0 1.0; 1.0 2.0]
    evals, evecs = eigen(C)
    ievecs = inv(evecs)
    Λ, A, iA = make_diag_Tesnor(evals, evecs, ievecs)
    @test 
end

Test Summary:        | Total  Time
make_diag_Tensorのテスト |     0  0.0s


Test.DefaultTestSet("make_diag_Tensorのテスト", Any[], 0, false, false, true, 1.753432781294533e9, 1.753432781294639e9, false, "In[194]")

In [156]:
@show A
@show iA
;

A = ITensor ord=2
Dim 1: (dim=2|id=990|"eig_1")
Dim 2: (dim=2|id=828|"old_1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 -0.7071067811865475  0.7071067811865475
  0.7071067811865475  0.7071067811865475
iA = ITensor ord=2
Dim 1: (dim=2|id=922|"old_2")
Dim 2: (dim=2|id=292|"eig_2")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 -0.7071067811865476  0.7071067811865476
  0.7071067811865476  0.7071067811865476


In [150]:
new_1 = Index(2, "new = 1")
new_2 = Index(2, "new = 2")
new1_list = [new_1]
new2_list = [new_2]
C = [2.0 1.0; 1.0 2.0]
evals, evecs = eigen(C)
ievecs = inv(evecs)
Λ, A, iA = make_diag_Tesnor(evals, evecs, ievecs)
inds(A)[1]
inds(A)[2]
A_ = replaceinds(A, (inds(A)[1], inds(A)[2]) => (inds(A)[1], new1_list..., ))
inds(A_)[1]
inds(A_)[2]
A*iA
;

In [203]:
T = randomITensor(ind1, ind2)

ITensor ord=2 (dim=4|id=197|"new=1") (dim=2|id=157|"new=2")
NDTensors.Dense{Float64, Vector{Float64}}

In [201]:
ind1 = Index(4, "new = 1")
ind2 = Index(2, "new = 2")
ind3 = Index(2, "new = 3")
@show indlist = (ind2, ind3)
@show diagITensor([1.0, 1.0], ind1, indlist)
;

indlist = (ind2, ind3) = ((dim=2|id=157|"new=2"), (dim=2|id=724|"new=3"))
diagITensor([1.0, 1.0], ind1, indlist) = ITensor ord=3
Dim 1: (dim=4|id=197|"new=1")
Dim 2: (dim=2|id=157|"new=2")
Dim 3: (dim=2|id=724|"new=3")
NDTensors.Diag{Float64, Vector{Float64}}
 4×2×2
[:, :, 1] =
 1.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

[:, :, 2] =
 0.0  0.0
 0.0  1.0
 0.0  0.0
 0.0  0.0


In [ ]:
#tagの張り替えの練習
new_1 = Index(4, "new = 1")
new_2 = Index(2, "new = 2")
new_3 = Index(2, "new = 3")
new_4 = Index(2, "new = 4")
old_inds = (new_1, new_2)
new_inds = (new_3, new_4, new_2)  
T = ITensor(new_1, new_2)
T_ = replaceinds(T, old_inds => new_inds)

LoadError: DimensionMismatch: arrays could not be broadcast to a common size: a has axes Base.OneTo(2) and b has axes Base.OneTo(3)

In [ ]:
new_1 = Index(2, "new = 1")
new_2 = Index(2, "new = 2")
new_3 = Index(2, "new = 3")
new_4 = Index(2, "new = 4")
new1_list = [new_1, new_2]
new2_list = [new_3, new_4]
C = [2.0 1.0 2; 1.0 2.0]
evals, evecs = eigen(C)
ievecs = inv(evecs)
Λ, A, iA = make_diag_Tesnor(evals, evecs, ievecs)
@show inds(A)[1]
@show inds(A)[2]
A_ = replaceinds(A, (inds(A)[1], inds(A)[2]) => (inds(A)[1], new1_list..., ))
@show inds(A_)[1]
@show inds(A_)[2]
;

In [ ]:
@testset "make_diag_Tensorのテスト" begin
    new_1 = Index(2, "new = 1")
    new_2 = Index(2, "new = 2")
    new1_list =[new_1]
    new2_list = [new_2]
    C = [2.0 1.0; 1.0 2.0]
    evals, evecs = eigen(C)
    ievecs = inv(evecs)
    Λ, A, iA = make_diag_Tesnor(evals, evecs, ievecs, new1_list, new2_list)
end

In [96]:
@show iA * T * A

iA * T * A = ITensor ord=2
Dim 1: (dim=2|id=110|"old=2")
Dim 2: (dim=2|id=212|"old=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 2.0  1.0
 1.0  2.0


ITensor ord=2 (dim=2|id=110|"old=2") (dim=2|id=212|"old=1")
NDTensors.Dense{Float64, Vector{Float64}}

In [99]:
C = [2.0 1.0; 1.0 2.0]

2×2 Matrix{Float64}:
 2.0  1.0
 1.0  2.0

In [ ]:
function diag(C::ITensor,)

$$ 角転送行列の対角化

In [ ]:
β = 1.0
J = 1.0

1.0

In [ ]:
#内部テンソルと境界条件を作る。
@show σ = CTMRG.make_2D_index_σtags(1)
@show s = CTMRG.make_2D_index_stags(1)
T_list = CTMRG.make_inner_tensors(1, β, J, s, σ)
;

σ = CTMRG.make_2D_index_σtags(1) = Index[(dim=2|id=629|"i=1,j=1,σ") (dim=2|id=317|"i=2,j=1,σ"); (dim=2|id=267|"i=1,j=2,σ") (dim=2|id=337|"i=2,j=2,σ"); (dim=2|id=716|"i=1,j=3,σ") (dim=2|id=855|"i=2,j=3,σ")]
s = CTMRG.make_2D_index_stags(1) = Index[(dim=2|id=662|"i=1,j=1,s") (dim=2|id=810|"i=2,j=1,s") (dim=2|id=557|"i=3,j=1,s"); (dim=2|id=537|"i=1,j=2,s") (dim=2|id=164|"i=2,j=2,s") (dim=2|id=378|"i=3,j=2,s")]


In [ ]:
#初期化テンソルを作る関数
N = 4
T_list = Array{ITensor}(undef, N, N)

for i in 1:N, j in 1:N
    T_list[i, j] = ITensor(true)
end
function setup_tensors_2D_initial(L)
    T_list = Array{ITensor}(undef, 2*L, 2*L)
    for i in 1:2*L, j in 1:2*L
        T_list[i, j] = ITensor(true)
    end
    return T_list
end

setup_tensors_2D_initial (generic function with 1 method)

In [ ]:
 @testset "setup_tensors_2D_initial" begin
    L = 10
    T_list = setup_tensors_2D_initial(L)
    @test length(T_list) == 2*L * 2*L
    @test all(ITensor(true) == T_list[i, j] for i in 1:2*L, j in 1:2*L)
end

Test Summary:            | Pass  Total  Time
setup_tensors_2D_initial |    2      2  0.0s


Test.DefaultTestSet("setup_tensors_2D_initial", Any[], 2, false, false, true, 1.753417900583083e9, 1.753417900606943e9, false, "In[23]")

In [ ]:
#境界条件との縮約をとった形をあらわす。
L = 5
T1_list = setup_tensors_2D_initial(L)
T_list, Dup_list, Ddown_list, L_list, R_list = CTMRG.setup_tensors_2D(L, β, J)
;

In [ ]:
T1_list[1, 2]
order(T1_list[1, 2]) 

0

In [ ]:
#境界条件との縮約をとった関数を定義する関数。
function 
for (i, Ddown) in enumerate(Ddown_list)
    T1_list[i,1] = T_list[i,1] * Ddown
end
for (i, L) in enumerate(L_list)
    ord = order(T1_list[1,i])
    if ord == 0
        T1_list[1,i] = T_list[1,i] * L
    else
        T1_list[1,i] = T1_list[1,i] * L
    end
end
for (i, Dup) in enumerate(Dup_list)
    ord = order(T1_list[i,2*L])
    if ord == 0
        T1_list[i,2*L] = T_list[i,2*L] * Dup
    else
        T1_list[i,2*L] = T1_list[i,2*L] * Dup
    end
end
for (i, R) in enumerate(R_list)
    ord = order(T1_list[2*L,i])
    if ord == 0
        T1_list[2*L,i] = T_list[2*L,i] * R
    else
        T1_list[2*L,i] = T1_list[2*L,i] * R
    end
end

In [ ]:
T1_list[1,1]

ITensor ord=2 (dim=2|id=425|"i=1,j=2,σ") (dim=2|id=117|"i=2,j=1,s")
NDTensors.Dense{Float64, Vector{Float64}}

In [ ]:
T = T1_list[2,1]
(i1, i2) = inds(T)
i1

(dim=2|id=146|"i=2,j=2,σ")

In [ ]:
A_mat = array(permute(T, (i1, i2)))
#eigvals, eigvecs = eigen(A_mat, i1, i2)

LoadError: In `permute(::ITensor, inds...)`, the input ITensor has indices: 

((dim=2|id=146|"i=2,j=2,σ"), (dim=2|id=117|"i=2,j=1,s"), (dim=2|id=236|"i=3,j=1,s"))

but the desired Index ordering is: 

((dim=2|id=146|"i=2,j=2,σ"), (dim=2|id=117|"i=2,j=1,s"))

In [ ]:
@show eigvals(A_mat)
@show eigvecs(A_mat)

eigvals(A_mat) = [0.9813491296916298, 54.61680090345261]
eigvecs(A_mat) = [0.018647627255526746 -0.9998261178813739; -0.9998261178813739 -0.018647627255526746]


2×2 Matrix{Float64}:
  0.0186476  -0.999826
 -0.999826   -0.0186476

In [ ]:
@which eigen

LinearAlgebra

In [ ]:
storage(T)

4-element NDTensors.Dense{Float64, Vector{Float64}}:
 54.598150033144236
  1.0
  1.0
  1.0

In [ ]:
M = dense(T)

ITensor ord=2 (dim=2|id=325|"i=2,j=2,σ") (dim=2|id=431|"i=2,j=1,s")
NDTensors.Dense{Float64, Vector{Float64}}

In [ ]:
eigvals, eigvecs = eigen(M)

LoadError: Number of left and right indices must add up to total number of indices

In [112]:
# 1. 線形化されたインデックス
i_lin = Index(dim, "i_lin")
j_lin = Index(dim, "j_lin")

# 2. 線形なテンソルを作る
A = ITensor(i_lin, j_lin)
for i in 1:dim, j in 1:dim
    A[i_lin => i, j_lin => j] = mat[i, j]
end

LoadError: MethodError: no method matching Index(::typeof(dim), ::String)
The type `Index` exists, but no method is defined for this combination of argument types when trying to construct it.

[0mClosest candidates are:
[0m  Index(::Any, ::T, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m) where T
[0m[90m   @[39m [35mITensors[39m [90m~/.julia/packages/ITensors/elsyT/src/[39m[90m[4mindex.jl:48[24m[39m
[0m  Index([91m::Vector{Pair{QN, Int64}}[39m, ::Any; dir, plev)
[0m[90m   @[39m [35mITensors[39m [90m~/.julia/packages/ITensors/elsyT/src/qn/[39m[90m[4mqnindex.jl:148[24m[39m
[0m  Index([91m::Number[39m, ::Union{ITensors.TagSets.GenericTagSet{BitIntegers.UInt256, 4}, AbstractString}; plev)
[0m[90m   @[39m [35mITensors[39m [90m~/.julia/packages/ITensors/elsyT/src/[39m[90m[4mindex.jl:100[24m[39m
[0m  ...


In [ ]:


# 3. 新しい多次元インデックスを定義
i1 = Index(2, "i1")
i2 = Index(2, "i2")
j1 = Index(2, "j1")
j2 = Index(2, "j2")

# 4. 線形Index → 多次元Indexの置き換え
A_new = replaceinds(A, (i_lin => (i1, i2), j_lin => (j1, j2)))


LoadError: MethodError: no method matching Index(::typeof(dim), ::String)
The type `Index` exists, but no method is defined for this combination of argument types when trying to construct it.

[0mClosest candidates are:
[0m  Index(::Any, ::T, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m) where T
[0m[90m   @[39m [35mITensors[39m [90m~/.julia/packages/ITensors/elsyT/src/[39m[90m[4mindex.jl:48[24m[39m
[0m  Index([91m::Vector{Pair{QN, Int64}}[39m, ::Any; dir, plev)
[0m[90m   @[39m [35mITensors[39m [90m~/.julia/packages/ITensors/elsyT/src/qn/[39m[90m[4mqnindex.jl:148[24m[39m
[0m  Index([91m::Number[39m, ::Union{ITensors.TagSets.GenericTagSet{BitIntegers.UInt256, 4}, AbstractString}; plev)
[0m[90m   @[39m [35mITensors[39m [90m~/.julia/packages/ITensors/elsyT/src/[39m[90m[4mindex.jl:100[24m[39m
[0m  ...
